In [ ]:
stack_name = 'statusapi'

In [ ]:
cf_bucket = 'ds-deploy-us-east-1'
template_name = 'statusapi-deploy.yml'
smssub_zip = 'statusapi.zip'

In [ ]:
%%bash -s "$template_name" "$smssub_zip" "$cf_bucket"
make
aws s3 cp ./$1 s3://$3
aws s3 cp ./$2 s3://$3

In [ ]:
%%bash -s "$template_name" "$cf_bucket"
aws cloudformation package \
  --template-file $1 \
  --output-template-file serverless-output.yml  \
  --s3-bucket $2

In [ ]:
%%bash -s "$stack_name" "$cf_bucket"
aws cloudformation deploy \
    --template-file serverless-output.yml \
    --stack-name $1 \
    --capabilities CAPABILITY_IAM \
    --parameter-overrides CodeBucketName=$2

In [ ]:
import boto3
client = boto3.client('cloudformation')

response = client.describe_stack_resources(
    StackName=stack_name
)


restapi_id = ''

resources = response['StackResources']
for r in resources:
    rid = r['LogicalResourceId']
    if rid == 'ServerlessRestApi':
        restapi_id = r['PhysicalResourceId']
    
        
print restapi_id


In [ ]:
# form endpoint
import os
region = os.environ['AWS_DEFAULT_REGION']

endpoint = 'https://{}.execute-api.{}.amazonaws.com/{}/instances'.format(restapi_id, region, 'Prod')
print endpoint

In [ ]:
%%bash -s "$endpoint"
curl -v $1

In [ ]:
ddb_client = boto3.client('dynamodb')

response = ddb_client.query(
    TableName=tablename,
    KeyConditions={
        'InstanceID': {
            'AttributeValueList':[
                {'S':'W024317-19JAN18'}
            ],
            'ComparisonOperator': 'EQ'
        }
    }
)

print response

## Clean Up

In [ ]:
print 'destroy {}'.format(smssub_stack)
response = client.delete_stack(
    StackName=smssub_stack
)

print response

waiter = client.get_waiter('stack_delete_complete')
print 'waiting for destroy of {}...'.format(smssub_stack)
waiter.wait(
    StackName=smssub_stack
)

print 'Stack {} destroyed'.format(smssub_stack)